In [47]:
import math
import operator
import numpy as np
import pandas as pd


def f1(x):
    return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2


def f1_deriv(x, index):
    return {
        1: 2 * (200 * x[0] ** 3 - 200 * x[0] * x[1] + x[0] - 1),
        2: 200 * (x[1] - x[0] ** 2)
    }.get(index)


def f2(x):
    return (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2


def f2_deriv(x, index):
    return {
        1: 2 * (2 * x[0] ** 3 - 2 * x[0] * x[1] + x[0] - 1),
        2: 2 * (x[1] - x[0] ** 2)
    }.get(index)


def f3(x):
    return (1.5 - x[0] * (1 - x[1])) ** 2 + (2.25 - x[0] * (1 - x[1] ** 2)) ** 2 + (2.625 - x[0] * (1 - x[1] ** 3)) ** 2


def f3_deriv(x, index):
    return {
        1: 2 * x[0] * (
                x[1] ** 6 + x[1] ** 4 - 2 * x[1] ** 3 - x[1] ** 2 - 2 * x[1] + 3) + 5.25 * x[1] ** 3 + 4.5 * x[1] ** 2 + 3 * x[1] - 12.75,
        2: x[0] * (x[0] * (6 * x[1] ** 5 + 4 * x[1] ** 3 - 6 * x[1] ** 2 - 2 * x[1] - 2) + 15.75 * x[1] ** 2 + 9 * x[1] + 3)
    }.get(index)


def f4(x):
    return (x[0] + x[1]) ** 2 + 5 * (x[2] - x[3]) ** 2 + (x[1] - 2 * x[2]) ** 4 + 10 * (x[0] - x[3]) ** 4


def f4_deriv(x, index):
    return {
        1: 2 * (20 * (x[0] - x[3]) ** 3 + x[0] + x[1]),
        2: 2 * (x[0] + 2 * (x[1] - 2 * x[2]) ** 3 + x[1]),
        3: 10 * (x[2] - x[3]) - 8 * (x[1] - 2 * x[2]) ** 3,
        4: 10 * (-4 * (x[0] - x[3]) ** 3 + x[3] - x[2])
    }.get(index)


def getFunc(index):
    return {1: f1,
            2: f2,
            3: f3,
            4: f4
            }.get(index)


def getFuncDeriv(index):
    return {1: f1_deriv,
            2: f2_deriv,
            3: f3_deriv,
            4: f4_deriv
            }.get(index)

def getFuncVariableNumber(index):
    return {1: 2,
            2: 2,
            3: 2,
            4: 4}.get(index)

In [48]:
def goldenRatio(a, b, func, eps):
    PHI = (1 + np.sqrt(5)) / 2
    len_prev = 0
    x1 = b - (b - a) / PHI
    x2 = a + (b - a) / PHI
    f1 = func(x1)
    f2 = func(x2)
    counter = 0
    while abs(b - a) > eps: 
        counter += 1
        len_prev = b - a
        if f1 < f2:
            b = x2
            f2, x2 = f1, x1
            x1 = b - (b - a) / PHI
            f1 = func(x1)
        else:
            a = x1
            f1, x1 = f2, x2
            x2 = a + (b - a) / PHI
            f2 = func(x2)
    return (a + b) / 2

In [49]:
def steepestDescent(func_index, eps):
    func = getFunc(func_index)
    func_deriv = getFuncDeriv(func_index)
    variable_number = getFuncVariableNumber(func_index)
    x = [0 for i in range(variable_number)]
    points = [(x, func(x))]
    while True:
      grad = [func_deriv(x, i) for i in range(1, variable_number + 1)]
      new_x_func = lambda step: list(map(operator.sub, x, [step * grad[i] for i in range(len(grad))]))
      step_func = lambda step: func(new_x_func(step))
      result_step = goldenRatio(0, 10, step_func, eps)
      x = new_x_func(result_step)
      points.append((x, func(x)))
      if abs(points[-1][1] - points[-2][1]) < eps:
        break;
    print(points[-1])

In [50]:
for i in range (1, 5):
    steepestDescent(i, 10 ** -9)

([0.999369497486081, 0.9987362381883463], 3.9852839178054377e-07)
([0.9999427194174707, 0.999885441848697], 3.281065134966883e-09)
([2.999776690682426, 0.4999446214009225], 7.983748205856503e-09)
([0.0, 0.0, 0.0, 0.0], 0.0)


In [51]:
def calculateX_hat(x1, x2, f1, f2):
    a = (f2 - f1) / (x2 - x1)
    b = f1 - x1 * a
    return - (b / a)

In [52]:
def writeRowBrent(data, a, c, x, w, v, u, f_x, f_w, f_v, f_u):
    data['a'].append(a)
    data['c'].append(c)
    data['x'].append(x)
    data['w'].append(w)
    data['v'].append(v)
    data['u'].append(u)
    data['f_x'].append(f_x)
    data['f_w'].append(f_w)
    data['f_v'].append(f_v)
    data['f_u'].append(f_u)


def uMerge(a, c, x, w, v, g, eps, func_index):
    func = getFunc(func_index)
    func_deriv = getFuncDeriv(func_index)
    variable_number = getFuncVariableNumber(func_index)
    f_x_deriv = f_w_deriv = f_v_deriv = np.array([func_deriv(x, i) for i in range(1, variable_number + 1)])    
    u1 = None
    if (x != w).any() and (f_x_deriv != f_w_deriv).any():
        u1 = calculateX_hat(x, w, f_x_deriv, f_w_deriv)
        for i in range(variable_number):
            if u1[i] >= a[i] + eps and u1[i] <= c[i] - eps and abs(u1[i] - x[i]) < g[i] / 2:
                u1[i] = u1[i]
            else: 
                u1[i] = None
    u2 = None
    if (x != v).any() and (f_x_deriv != f_v_deriv).any():
        u2 = calculateX_hat(x, v, f_x_deriv, f_v_deriv)
        for i in range(variable_number):
            if u2[i] >= a[i] + eps and u2[i] <= c[i] - eps and abs(u2[i] - x[i]) < g[i] / 2:
                u2[i] = u2[i]
            else:
                u2[i] = None
    u = u1 if u1 is not None else np.array([0 for i in range(variable_number)])
    for i in range(variable_number):
        if u1 is not None and u2 is not None:
            if u[i] is not None and u2[i] is not None:
                if abs(u[i] - x[i]) > abs(u2[i] - x[i]):
                    u[i] = u2[i]
            else:
                if u[i] is None and u2[i] is not None:
                    u[i] = u2[i]
            if u[i] is None:
                if f_x_deriv[i] > 0:
                    u[i] = (a[i] + x[i]) / 2
                else:
                    u[i] = (x[i] + c[i]) / 2
        else:
            if f_x_deriv[i] > 0:
                u[i] = (a[i] + x[i]) / 2
            else:
                u[i] = (x[i] + c[i]) / 2            
        if abs(u[i] - x[i]) < eps:
            u[i] = x[i] + np.sign(u[i] - x[i]) * eps
    return u


def brent(a, c, eps, func_index):
    func = getFunc(func_index)
    func_deriv = getFuncDeriv(func_index)
    variable_number = getFuncVariableNumber(func_index)
    data = {'a' : [], 'c' : [], 'x' : [], 'w' : [], 'v' : [],
            'u': [], 'f_x' : [], 'f_w' : [], 'f_v' : [], 'f_u' : []} 
    x = w = v = (a + c) / 2
    f_x = f_w = f_v = func(x)
    f_x_deriv = f_w_deriv = f_v_deriv = [func_deriv(x, i) for i in range(1, variable_number + 1)]
    d = e = c - a
    counter = 0
    while True:
        counter += 1
        g, e = e, d     
        u = uMerge(a, c, x, w, v, g, eps, func_index)
        d = np.array([abs(x[i] - u[i]) for i in range(variable_number)])
        f_u = func(u)
        f_u_deriv = [func_deriv(u, i) for i in range(1, variable_number + 1)]
        writeRowBrent(data, a, c, x, w, v, u, f_x, f_w, f_v, f_u)
        if f_u <= f_x:
            for i in range(variable_number):
                if u[i] >= x[i]:
                    a[i] = x[i]
                else:
                    c[i] = x[i]
            v, w, x = w, x, u 
            f_v, f_w, f_x = f_w, f_x, f_u
            f_v_deriv, f_w_deriv, f_x_deriv = f_w_deriv, f_x_deriv, f_u_deriv
        else:
            for i in range(variable_number):
                if u[i] >= x[i]:
                    c[i] = u[i]
                else:
                    a[i] = u[i]
                if f_u <= f_w or (w == x).all():
                    v, w = w, u 
                    f_v, f_w = f_w, f_u
                    f_v_deriv, f_w_deriv = f_w_deriv, f_u_deriv
                elif f_u <= f_v or (v == x).all() or (v == w).all():
                    v = u
                    f_v = f_u
                    f_v_deriv = f_u_deriv
        if counter != 1:
            out_condition = np.array([abs(prev_u[i] - u[i]) for i in range(variable_number)])
            print(a, c)
            if (out_condition < eps).all():
                break
        prev_u = u
    df = pd.DataFrame(data)
    with pd.ExcelWriter(f'brent_{func_index}.xlsx') as writer:
        df.to_excel(writer)
    return counter, (c + a) / 2

In [53]:
print(brent(np.array([-5, -5, -5, -5]), np.array([5, 5, 5 ,5]), 10 ** -5, 4))

[-5 -5 -5 -5] [1 1 1 1]
[0 0 0 0] [1 1 1 1]
[0 0 0 0] [1 1 1 1]
(4, array([0.5, 0.5, 0.5, 0.5]))
